In [23]:
import sqlite3
import os
from pathlib import Path
from anthropic import Anthropic
from datetime import datetime
 
REPO_PATH = Path("/Users/koen/workspace/agentic-syftbox")
SCREENPIPE_PATH = REPO_PATH / "data" / "screenpipe" / "db2.sqlite" 


In [24]:
# Connect to the SQLite database
conn = sqlite3.connect(SCREENPIPE_PATH)
conn.row_factory = sqlite3.Row

In [25]:
cursor = conn.cursor()
# Query the audio_transcriptions table
cursor.execute("SELECT * FROM audio_transcriptions")
rows = [dict(x) for x in cursor.fetchall()]
fragments = []

meetings = []
current_meeting_rows = []
if len(rows) > 0:
    previous_start_date = datetime.fromisoformat(rows[0]["timestamp"])

    for row in rows:
        speaker_id = row["speaker_id"]
        transcription = row["transcription"]
        timestamp = row["timestamp"]
        
        chunk_datetime = datetime.fromisoformat(timestamp)
        time_diff = (chunk_datetime - previous_start_date).total_seconds() / 60
        if time_diff > 30:
            if current_meeting_rows:
                meetings.append(current_meeting_rows)
            current_meeting_rows = [row]
            previous_start_date = chunk_datetime
        else:
            current_meeting_rows.append(row)
    meetings.append(current_meeting_rows)
        
        


table meeting_meta

meeting_id | filename

table meeting_chunks

meeting_i | chunkid

```

CREATE TABLE IF NOT EXISTS meeting_meta (
    meeting_id INTEGER PRIMARY KEY,
    filename TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS meeting_chunks (
    meeting_id INTEGER,
    chunkid INTEGER,
    date_unix INTEGER,
    text TEXT,
    FOREIGN KEY (meeting_id) REFERENCES meeting_meta(meeting_id),
    PRIMARY KEY (meeting_id, chunkid)
);
```


Meetings CTE
| Meeting id | filename | full text | start date | end date


```
WITH meetings AS (
  SELECT
    mc.meeting_id,
    mm.filename,
    MIN(mc.date) AS start_date,
    MAX(mc.date) AS end_date,
    GROUP_CONCAT(mc.text, ' ') AS full_text
  FROM (
    SELECT * FROM meeting_chunks ORDER BY date, chunkid
  ) mc
  JOIN meeting_meta mm ON mc.meeting_id = mm.meeting_id
  GROUP BY mc.meeting_id
)
SELECT *
FROM meetings;
```

In [28]:
for chunks in meetings:
    date = chunks[0]["timestamp"]
    print(f"meeting {date}")
    for chunk in chunks:
        print(chunk["audio_chunk_id"])
    
    print()
    

meeting 2025-06-03T14:30:48.452985+00:00
1
2

meeting 2025-06-05T13:00:36.357322+00:00
3
4
4
5
5
6
7

meeting 2025-06-05T14:08:44.519520+00:00
8

